In [ ]:
import logging
import asyncio
import ib_async
from ezib_async import ezIBAsync, util

In [ ]:
util.logToConsole('DEBUG')
logging.getLogger('ib_async').setLevel('CRITICAL')

In [ ]:
# initialize ezIBAsync
ezib = ezIBAsync()

# connect to IB (7496/7497 = TWS, 4001 = IBGateway)
# await ezib.connectAsync(ibclient=101, ibhost="localhost", ibport=4001, account='U13792768')
# ezib.disconnect()

await ezib.connectAsync(ibclient=101, ibhost="localhost", ibport=4001, account='U9860850')

In [ ]:

contracts = await asyncio.gather(
    # option contracts
    ezib.createOptionContract("AAPL", expiry="20250530", strike=200, otype="P"),
    
    # futures contracts
    # ezib.createFuturesContract("ES", expiry="202506", exchange="CME"),
    # ezib.createContract(("CL", "FUT", "NYMEX", "USD", "202506", 0.0, "")),
    
    # forex contracts
    # ezib.createForexContract("EUR", currency="USD")
    
)

In [ ]:
# request market data for all created contracts
await ezib.requestMarketData()

In [ ]:
[attr for attr in dir(ib_async.Ticker.modelGreeks) if not attr.startswith('_')]

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[ezib.contractString(c) for c in ezib.contracts.values() if c.secType in ("OPT", "FOP")],
    # columns=['iv', 'volume'])
    # columns=['last_undPrice', 'model_undPrice', 'ask_undPrice', 'bid_undPrice'])
    columns=['last', 'lastSize', 'price', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta']) 
    # columns=['ask', 'askSize', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta'])
    # columns=['bid', 'bidSize', 'price', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta']) 
    # columns=['price', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta'])

def onOptionsTickerUpdate(tickers):
    for t in tickers:
        df.loc[ezib.contractString(t.contract)] = (
            # t.impliedVolatility, t.volume)
            # t.lastGreeks.undPrice, t.modelGreeks.undPrice, t.askGreeks.undPrice, t.bidGreeks.undPrice)
            t.last, t.lastSize, t.lastGreeks.optPrice, t.lastGreeks.impliedVol, t.lastGreeks.undPrice, t.lastGreeks.delta, t.lastGreeks.gamma, t.lastGreeks.vega, t.lastGreeks.theta)
            # t.ask, t.askSize, t.askGreeks.impliedVol, t.askGreeks.undPrice, t.askGreeks.delta, t.askGreeks.gamma, t.askGreeks.vega, t.askGreeks.theta)
            # t.bid, t.bidSize, t.bidGreeks.optPrice, t.bidGreeks.impliedVol, t.bidGreeks.undPrice, t.bidGreeks.delta, t.bidGreeks.gamma, t.bidGreeks.vega, t.bidGreeks.theta)
            # t.modelGreeks.optPrice, t.modelGreeks.impliedVol, t.modelGreeks.undPrice, t.modelGreeks.delta, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta)
        clear_output(wait=True)
    display(df)
    # print(t)
    
ezib.pendingOptionsTickersEvent += onOptionsTickerUpdate
await asyncio.sleep(15)
ezib.pendingOptionsTickersEvent -= onOptionsTickerUpdate

In [ ]:
ezib.contracts

In [ ]:
ticker = ezib.ib.ticker(ezib.contracts[1])

In [ ]:
df = util.df([ticker])

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[ezib.contractString(c) for c in ezib.contracts.values() if c.secType in ("OPT", "FOP")],
    # columns=['iv', 'volume'])
    # columns=['last_undPrice', 'model_undPrice', 'ask_undPrice', 'bid_undPrice'])
    columns=['last', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta']) 
    # columns=['ask', 'askSize', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta'])
    # columns=['bid', 'bidSize', 'price', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta']) 
    # columns=['price', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta'])

def onOptionsTickerUpdate(tickers):
    for t in tickers:
        df.loc[ezib.contractString(t.contract)] = (
            # t.impliedVolatility, t.volume)
            # t.lastGreeks.undPrice, t.modelGreeks.undPrice, t.askGreeks.undPrice, t.bidGreeks.undPrice)
            t.modelGreeks.optPrice, t.modelGreeks.impliedVol, t.modelGreeks.undPrice, t.modelGreeks.delta, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta)
            # t.modelGreeks.delta * t.modelGreeks.undPrice * 100)
            # t.ask, t.askSize, t.askGreeks.impliedVol, t.askGreeks.undPrice, t.askGreeks.delta, t.askGreeks.gamma, t.askGreeks.vega, t.askGreeks.theta)
            # t.bid, t.bidSize, t.bidGreeks.optPrice, t.bidGreeks.impliedVol, t.bidGreeks.undPrice, t.bidGreeks.delta, t.bidGreeks.gamma, t.bidGreeks.vega, t.bidGreeks.theta)
            # t.modelGreeks.optPrice, t.modelGreeks.impliedVol, t.modelGreeks.undPrice, t.modelGreeks.delta, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta)
        clear_output(wait=True)
    display(df)
    # print(t)
    
ezib.pendingOptionsTickersEvent += onOptionsTickerUpdate
await asyncio.sleep(30)
ezib.pendingOptionsTickersEvent -= onOptionsTickerUpdate


将不常用的的opiton，调整到子账户。 marketData += 与，opitonsData += 一起计算总体持仓

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[ezib.contractString(c) for c in ezib.contracts.values() if c.secType in ("OPT", "FOP")],
    # columns=['iv', 'volume'])
    # columns=['last_undPrice', 'model_undPrice', 'ask_undPrice', 'bid_undPrice'])
    columns=['postion', 'marketValue']) 
    # columns=['ask', 'askSize', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta'])
    # columns=['bid', 'bidSize', 'price', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta']) 
    # columns=['price', 'imp_vol', 'underlying', 'delta', 'gamma', 'vega', 'theta'])

def onPortfolioUpdate(portfolios):
    for p in portfolios:
        df.loc[ezib.contractString(p.contract)] = (
            p.postion, p.marketValue
            )
        clear_output(wait=True)
    display(df)
    # print(t)
    
ezib.ib.updatePortfolioEvent += onPortfolioUpdate
await asyncio.sleep(15)
ezib.ib.updatePortfolioEvent -= onPortfolioUpdate

In [ ]:
ezib.getPositions()['GLD20260116P00291000_OPT']['position']

In [ ]:
ticker = ezib.ib.ticker(ezib.contracts[10])

In [ ]:
ezib.contractString(ticker.contract)

In [ ]:
util.df(ezib.portfolios)

In [ ]:
# # cancel market data request & disconnect
ezib.cancelMarketData()
ezib.disconnect()